# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking the current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# use pandas to load the data
full_data_rows_list = [] 
column_name = ["artist","firstName","gender","itemInSession","lastName","length","level","location","sessionId","song","userId"]    
# for every filepath in the file path list 
for f in file_path_list:
    df = pd.read_csv(f)
    value = df[column_name].values
    for row in value:
        full_data_rows_list.append(row)
    
small_df = pd.DataFrame(data = full_data_rows_list, columns = column_name)
# drop rows with 'artist' column is null
small_df = small_df.dropna(subset=['artist'])
# check the number of rows in csv file
print(small_df.shape)
# save to a new csv file
small_df.to_csv('event_datafile_new.csv', index = False)



(6820, 11)


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [4]:
from cassandra.cluster import Cluster

# Build a session to establish connection and begin executing queries
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)
    

#### Create Keyspace

In [5]:
# create a keyspace named project2
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS project2 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [6]:
# connect the keyspace
try:
    session.set_keyspace('project2')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

Under this query, I will set the sessionid and iteminsession to be the partition key. Then insert the value from columns of sessionid, iteminsession, artist, song and length.

In [7]:
# set the sessionid and iteminsession to be the partition key
query = "CREATE TABLE IF NOT EXISTS session_history "
query = query + "(sessionid int, iteminsession int, artist text, song text, length float, PRIMARY KEY (sessionid,iteminsession ))"
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [8]:
# insert values
df = pd.read_csv('event_datafile_new.csv')
for _, line in df.iterrows():
    query = "INSERT INTO session_history(sessionid, iteminsession, artist, song, length)"
    query = query + "VALUES (%s, %s, %s, %s, %s)"
    session.execute(query, (int(line[8]),int(line[3]),line[0],line[9],float(line[5])))    

#### Do a SELECT to verify that the data have been inserted into each table

In [9]:
query = "SELECT artist, song, length FROM session_history WHERE sessionid = 338 AND iteminsession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist, row.song ,row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


I have set the primary key to sessionid and iteminsession, so I can use where clause to get my answer.

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

Under this query, I will used a composite key with userId,sessionId to be the partition key and itemInSession to be the clustering columns. Then insert the value from columns of userId, sessionId, itemInSession, artist, song, firstName, lastName.

In [10]:
# I used a composite key with userId,sessionId to be the partition key and itemInSession to be the clustering columns
query = "CREATE TABLE IF NOT EXISTS user_history "
query = query + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY ((userId,sessionId),itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
              

In [11]:
for i, line in df.iterrows():
    query = "INSERT INTO user_history(userId, sessionId, itemInSession, artist, song, firstName, lastName)"
    query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
    session.execute(query, (int(line[10]),int(line[8]),int(line[3]),line[0],line[9],line[1],line[4]))


#### Do a SELECT to verify that the data have been inserted into each table

In [12]:
query = "SELECT iteminsession, artist, song, firstName,lastName  FROM user_history WHERE userId = 10 AND sessionId = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist, row.song , row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


I can use where claus to filter userid and sessionid, because I have set the iteminsession to be the clustering column, the rows are sorted by this column

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

Under this query, I will used a composite key with song to be the partition key and firstname,lastname to be the clustering columns. Then insert the value from columns of song, firstName, lastName.
Because cassandra only allow unique rows, this will overwrite the duplicated row, I will get only 1 row for each person histened the song.

In [13]:
# I used a composite key with song to be the partition key and firstname,lastname to be the clustering columns
query = "CREATE TABLE IF NOT EXISTS played_history"
query = query + "(song text, firstname text, lastname text, PRIMARY KEY ((song), firstname, lastname))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [ ]:
for i, line in df.iterrows():
    query = "INSERT INTO played_history(song, firstName, lastName)"
    query = query + "VALUES (%s, %s, %s)"
    session.execute(query, (line[9],line[1],line[4]))


#### Do a SELECT to verify that the data have been inserted into each table

In [ ]:
query = "SELECT firstname,lastname FROM played_history WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.firstname, row.lastname)

We can see this query ordered by the second partition key.

### Drop the tables before closing out the sessions

In [7]:
## TO-DO: Drop the table before closing out the sessions
query = "DROP TABLE IF EXISTS session_history "
try:
    session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS user_history "
try:
    session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS played_history "
try:
    session.execute(query)
except Exception as e:
    print(e)


### Close the session and cluster connection¶

In [8]:
session.shutdown()
cluster.shutdown()